In [54]:
import pandas as pd
from IPython.utils import io
import xlsxwriter
from heuristics import Poly, Log, RM, heuristic
import helpers as help
import data_process
from chefboost import Chefboost as chef
import json
import numpy as np
from scipy.stats import wilcoxon

In [15]:
data_process.generate_fold_data()

balance-scale
breast-cancer
cars
flags
house-votes
lymphography
mushroom


C:\Users\etetteh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


nursery


C:\Users\etetteh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


tic-tac-toe


In [16]:
def get_results(df, row, heuristic_func):
    rule = heuristic(df, row, heuristic_func)
    rule_length = len(rule[:-1])
    rule_support = help.support(df, rule)

    return rule, help.format_rule(rule), rule_length, rule_support

In [17]:
def worksheet_header(worksheet):
    worksheet.write(0, 0, 'Poly')
    worksheet.write(1, 0, 'Rule')
    worksheet.write(1, 1, 'Length')
    worksheet.write(1, 2, 'Support')

    worksheet.write(0, 3, 'Log')
    worksheet.write(1, 3, 'Rule')
    worksheet.write(1, 4, 'Length')
    worksheet.write(1, 5, 'Support')
    
    worksheet.write(0, 6, 'RM')
    worksheet.write(1, 6, 'Rule')
    worksheet.write(1, 7, 'Length')
    worksheet.write(1, 8, 'Support')
    
    worksheet.write(0, 9, 'Decision Tree')
    worksheet.write(1, 9, 'Rule')
    worksheet.write(1, 10, 'Length')
    worksheet.write(1, 11, 'Support')
    
    worksheet.write(0, 12, 'Min length')
    worksheet.write(0, 13, 'Heuristic with min length')
    worksheet.write(0, 14, 'Max Support')
    worksheet.write(0, 15, 'Heuristic with max support')
    
    worksheet.write(0, 16, 'Global Optimization (Length)')
    worksheet.write(1, 16, 'Rule')
    worksheet.write(1, 17, 'Unique Rules')
    worksheet.write(1, 18, 'Min length')
    worksheet.write(1, 19, 'Max Support')
    
    worksheet.write(0, 20, 'Global Optimization (Support)')
    worksheet.write(1, 20, 'Rule')
    worksheet.write(1, 21, 'Unique Rules')
    worksheet.write(1, 22, 'Min length')
    worksheet.write(1, 23, 'Max Support')

In [35]:
def full_results_header(worksheet1, row=0,resultType='Length'):
    dtrow = row+2
    # merge_format = worksheet1.add_format({
    #     'align':    'center',
    #     'valign':   'vcenter',
    # })
    worksheet1.write(dtrow, 0, 'Dataset')
    worksheet1.write(dtrow, 1, 'No Attr')
    worksheet1.write(dtrow, 2, 'No rows')
    
    worksheet1.merge_range(row, 3, row, 9, 'Global optimization-'+resultType)
    worksheet1.write(dtrow, 3, 'No. of Unique rules')
    # Length
    worksheet1.merge_range(row+1, 4, row+1, 6, 'Length')
    worksheet1.write(dtrow, 4, 'Min')
    worksheet1.write(dtrow, 5, 'Avg')
    worksheet1.write(dtrow, 6, 'Max')
    # Support
    worksheet1.merge_range(row+1, 7, row+1, 9, 'Support')
    worksheet1.write(dtrow, 7, 'Min')
    worksheet1.write(dtrow, 8, 'Avg')
    worksheet1.write(dtrow, 9, 'Max')

    worksheet1.merge_range(row, 10, row, 16, 'Poly')
    worksheet1.write(dtrow, 10, 'No. of Unique rules')
    # Length
    worksheet1.merge_range(row+1, 11, row+1, 13, 'Length')
    worksheet1.write(dtrow, 11, 'Min')
    worksheet1.write(dtrow, 12, 'Avg')
    worksheet1.write(dtrow, 13, 'Max')
    # Support
    worksheet1.merge_range(row+1, 14, row+1, 16, 'Support')
    worksheet1.write(dtrow, 14, 'Min')
    worksheet1.write(dtrow, 15, 'Avg')
    worksheet1.write(dtrow, 16, 'Max')
    
    worksheet1.merge_range(row, 17, row, 23, 'Log')
    worksheet1.write(dtrow, 17, 'No. of Unique rules')
    # Length
    worksheet1.merge_range(row+1, 18, row+1, 20, 'Length')
    worksheet1.write(dtrow, 18, 'Min')
    worksheet1.write(dtrow, 19, 'Avg')
    worksheet1.write(dtrow, 20, 'Max')
    # Support
    worksheet1.merge_range(row+1, 21, row+1, 23, 'Support')
    worksheet1.write(dtrow, 21, 'Min')
    worksheet1.write(dtrow, 22, 'Avg')
    worksheet1.write(dtrow, 23, 'Max')
    
    worksheet1.merge_range(row, 24, row, 30, 'RM')
    worksheet1.write(dtrow, 24, 'No. of Unique rules')
    # Length
    worksheet1.merge_range(row+1, 25, row+1, 27, 'Length')
    worksheet1.write(dtrow, 25, 'Min')
    worksheet1.write(dtrow, 26, 'Avg')
    worksheet1.write(dtrow, 27, 'Max')
    # Support
    worksheet1.merge_range(row+1, 28, row+1, 30, 'Support')
    worksheet1.write(dtrow, 28, 'Min')
    worksheet1.write(dtrow, 29, 'Avg')
    worksheet1.write(dtrow, 30, 'Max')
    
    worksheet1.merge_range(row, 31, row, 37, 'Decision Tree')
    worksheet1.write(dtrow, 31, 'No. of Unique rules')
    # Length
    worksheet1.merge_range(row+1, 32, row+1, 34, 'Length')
    worksheet1.write(dtrow, 32, 'Min')
    worksheet1.write(dtrow, 33, 'Avg')
    worksheet1.write(dtrow, 34, 'Max')
    # Support
    worksheet1.merge_range(row+1, 35, row+1, 37, 'Support')
    worksheet1.write(dtrow, 35, 'Min')
    worksheet1.write(dtrow, 36, 'Avg')
    worksheet1.write(dtrow, 37, 'Max')

In [36]:
def full_results_accur_header(worksheet1, row=0,resultType='Length'):
    worksheet1.write(row, 0, 'Dataset')
    worksheet1.write(row, 1, 'Global accuracy-'+resultType)
    worksheet1.write(row, 2, 'Std')
    
    worksheet1.write(row, 3, 'Poly')
    worksheet1.write(row, 4, 'Std')
    
    worksheet1.write(row, 5, 'Log')
    worksheet1.write(row, 6, 'Std')
    
    worksheet1.write(row, 7, 'RM')
    worksheet1.write(row, 8, 'Std')
    
    worksheet1.write(row, 9, 'Decision tree')
    worksheet1.write(row, 10, 'Std')

In [46]:
def write_full_res_data(worksheet1, row,col, data,df=None):
    if isinstance(df, pd.DataFrame):
        # worksheet1.write(row, col, len(data))
        worksheet1.write(row, col, help.minSupport(data, df))
        worksheet1.write(row, col+1, help.avgSupport(data, df))
        worksheet1.write(row, col+2, help.maxSupport(data, df))
    else:
        worksheet1.write(row, col, len(data))
        worksheet1.write(row, col+1, help.minLength(data))
        worksheet1.write(row, col+2, help.avgLength(data))
        worksheet1.write(row, col+3, help.maxLength(data))

In [47]:
def decision_tree(data, algorithm='CART'):
    config = {'algorithm': algorithm}
    df = data.astype(str)
    with io.capture_output() as captured:
        model = chef.fit(df, config, target_label=df.columns[-1])

    file = open("outputs/rules/rules.json")
    rules = help.format_rules([json.load(file)])
    file.close()
    dff = []
    supp = []
    for r in rules[0]:
        d = help.get_matched_rule_in_data_frame(df, r)
        d['rule'] = [r for i in range(d.shape[0])]
        dff.append(d)
        supp.append(help.support(df,r))

    return pd.concat(dff, ignore_index=False).sort_index(), rules[0], supp

In [48]:
def train_decision(data):
    polyAllRules = []
    logAllRules = []
    rmAllRules = []
    dt, dtAllRules, dtAllSupp = decision_tree(data)
    globalAllRulesLen = []
    globalAllRulesSupport = []
    for row in list(data.index):
        poly = get_results(data, row, Poly)
        log = get_results(data, row, Log)
        rm = get_results(data, row, RM)
        # dtSupport = help.support(data, dt.loc[row, 'rule']) #decision tree support

        # Append all rules to list
        if poly[0] not in polyAllRules:
            polyAllRules.append(poly[0])
        if log[0] not in logAllRules:
            logAllRules.append(log[0])
        if rm[0] not in rmAllRules:
            rmAllRules.append(rm[0])

        allRules = [{'h': 'poly', 'rule': poly[0], 'support':poly[3]}, {'h': 'log', 'rule': log[0], 'support':log[3]}, {
            'h': 'rm', 'rule': rm[0], 'support':rm[3]}]

        minLen = help.minLength([r['rule'] for r in allRules])
        hWithMinLength = [r for r in allRules if len(r['rule'][:-1]) == minLen]

        maxSupport = max([poly[3], log[3], rm[3]])
        hWithMaxSupport = [r for r in allRules if r['support'] == maxSupport]

        globalRulesLen = [r['rule'] for r in hWithMinLength]
        for g in globalRulesLen:
            if g not in globalAllRulesLen:
                globalAllRulesLen.append(g)

        globalRulesSupport = [r['rule'] for r in hWithMaxSupport]
        for g2 in globalRulesSupport:
            if g2 not in globalAllRulesSupport:
                globalAllRulesSupport.append(g2)

    return polyAllRules, logAllRules, rmAllRules, dtAllRules, dtAllSupp, globalAllRulesLen, globalAllRulesSupport

In [49]:
def cross_val_res(data):
    outputs = []
    for d in data:
        output = train_decision(d['train'].astype(str))
        
        polyRules = output[0]
        polySupp = help.set_of_support(d['train'].astype(str), polyRules)
        
        logRules = output[1]
        logSupp = help.set_of_support(d['train'].astype(str), logRules)
        
        rmRules = output[2]
        rmSupp = help.set_of_support(d['train'].astype(str), rmRules)
        
        dtRules = output[3]
        dtSupp = output[4]
        
        globalRulesLen = output[5]
        globalRulesLenSupp = help.set_of_support(d['train'].astype(str), globalRulesLen)
        
        globalRulesSupport = output[6]
        globalRulesSupportSupp = help.set_of_support(d['train'].astype(str), globalRulesSupport)
        
        outputs.append({
            'poly': help.score(d['test'].astype(str),polyRules,polySupp)[0],
            'log': help.score(d['test'].astype(str),logRules,logSupp)[0],
            'rm': help.score(d['test'].astype(str),rmRules,rmSupp)[0],
            'dt': help.score(d['test'].astype(str),dtRules,dtSupp)[0],
            'global_Len': help.score(d['test'].astype(str),globalRulesLen,globalRulesLenSupp)[0],
            'global_Supp': help.score(d['test'].astype(str),globalRulesSupport, globalRulesSupportSupp)[0],
        })

    return outputs

In [52]:
datas = data_process.get_full_data('data')

In [53]:
workbook1 = xlsxwriter.Workbook('full_results.xlsx')
worksheet1 = workbook1.add_worksheet('result')
wksht_cross_val = workbook1.add_worksheet('cross_val_results')
full_results_header(worksheet1)
dcv = 0
for d, data in enumerate(datas):
    st=3
    workbook = xlsxwriter.Workbook(data['name']+'_results.xlsx')
    worksheet = workbook.add_worksheet('result')
    worksheet_header(worksheet)

    polyAllRules = []
    logAllRules = []
    rmAllRules = []
    dt, dtAllRules, dtAllSupp = decision_tree(data['data'])
    globalAllRulesLen = []
    globalAllRulesSupport = []
    for row in list(data['data'].index):
        start = 2
        poly = get_results(data['data'], row, Poly)
        log = get_results(data['data'], row, Log)
        rm = get_results(data['data'], row, RM)
        dtSupport = help.support(data['data'], dt.loc[row, 'rule'])

        # Append all rules to list
        if poly[0] not in polyAllRules:
            polyAllRules.append(poly[0])
        if log[0] not in logAllRules:
            logAllRules.append(log[0])
        if rm[0] not in rmAllRules:
            rmAllRules.append(rm[0])
        
        allRules = [{'h': 'poly', 'rule': poly[0], 'support':poly[3]}, {'h': 'log', 'rule': log[0], 'support':log[3]}, {
            'h': 'rm', 'rule': rm[0], 'support':rm[3]}]

        minLen = help.minLength([r['rule'] for r in allRules])
        hWithMinLength = [r for r in allRules if len(r['rule'][:-1]) == minLen]
        
        maxSupport = max([poly[3], log[3], rm[3]])
        hWithMaxSupport = [r for r in allRules if r['support'] == maxSupport]
        
        globalRulesLen = [r['rule'] for r in hWithMinLength]
        uniqueGlobalLen = [u for i, u in enumerate(globalRulesLen) if u not in globalRulesLen[:i]]
        ugLenLength = min([len(r[:-1]) for r in uniqueGlobalLen])
        ugLenSupport = max([e for e in hWithMinLength if any(e['rule'] == rule for rule in uniqueGlobalLen)], 
                           key=lambda x: x['support'])['support']
        for g in globalRulesLen:
            if g not in globalAllRulesLen:
                globalAllRulesLen.append(g)

        globalRulesSupport = [r['rule'] for r in hWithMaxSupport]
        uniqueGlobalSupport = [u for i, u in enumerate(globalRulesSupport) if u not in globalRulesSupport[:i]]
        ugSupLength = min([len(r[:-1]) for r in uniqueGlobalSupport])
        ugSupSupport = max([e for e in hWithMaxSupport if any(e['rule'] == rule for rule in uniqueGlobalSupport)],
                           key=lambda x: x['support'])['support']
        for g2 in globalRulesSupport:
            if g2 not in globalAllRulesSupport:
                globalAllRulesSupport.append(g2)

        worksheet.write(row+start, 0, poly[1])
        worksheet.write(row+start, 1, poly[2])
        worksheet.write(row+start, 2, poly[3])

        worksheet.write(row+start, 3, log[1])
        worksheet.write(row+start, 4, log[2])
        worksheet.write(row+start, 5, log[3])

        worksheet.write(row+start, 6, rm[1])
        worksheet.write(row+start, 7, rm[2])
        worksheet.write(row+start, 8, rm[3])

        worksheet.write(row+start, 9, help.format_rule(dt.loc[row, 'rule']))
        worksheet.write(row+start, 10, len(dt.loc[row, 'rule'][:-1]))
        worksheet.write(row+start, 11, dtSupport)
        
        worksheet.write(row+start, 12, minLen)
        worksheet.write(row+start, 13, ', '.join([str(r['h']) for r in hWithMinLength]))
        worksheet.write(row+start, 14, maxSupport)
        worksheet.write(row+start, 15, ', '.join([str(r['h']) for r in hWithMaxSupport]))

        worksheet.write(row+start, 16, str([help.format_rule(r) for r in globalRulesLen]))
        worksheet.write(row+start, 17, str([help.format_rule(r) for r in uniqueGlobalLen]))
        worksheet.write(row+start, 18, ugLenLength)
        worksheet.write(row+start, 19, ugLenSupport)
        
        worksheet.write(row+start, 20, str([help.format_rule(r) for r in globalRulesSupport]))
        worksheet.write(row+start, 21, str([help.format_rule(r) for r in uniqueGlobalSupport]))
        worksheet.write(row+start, 22, ugSupLength)
        worksheet.write(row+start, 23, ugSupSupport)

    workbook.close()
    worksheet1.write(d+st, 0, data['name'])
    worksheet1.write(d+st, 1, len(list(data['data'].columns)[:-1]))
    worksheet1.write(d+st, 2, len(list(data['data'].index)))
    
    write_full_res_data(worksheet1, d+st, 3, globalAllRulesLen) # Global optimization-Length
    write_full_res_data(worksheet1, d+st, 7, globalAllRulesLen, data['data']) # Global optimization-Length
    write_full_res_data(worksheet1, d+st, 10, polyAllRules) # Poly-Length
    write_full_res_data(worksheet1, d+st, 14, polyAllRules, data['data']) # Poly-Length
    write_full_res_data(worksheet1, d+st, 17, logAllRules) # Log-Length
    write_full_res_data(worksheet1, d+st, 21, logAllRules, data['data']) # Log-Length
    write_full_res_data(worksheet1, d+st, 24, rmAllRules) # RM-Length
    write_full_res_data(worksheet1, d+st, 28, rmAllRules, data['data']) # RM-Length
    write_full_res_data(worksheet1, d+st, 31, dtAllRules) #Decision tree-Length
    write_full_res_data(worksheet1, d+st, 35, dtAllRules, data['data']) #Decision tree-Length

    full_results_header(worksheet1, len(datas)+2, 'Support')
    st = len(datas)+6

    worksheet1.write(d+st, 0, data['name'])
    worksheet1.write(d+st, 1, len(list(data['data'].columns)[:-1]))
    worksheet1.write(d+st, 2, len(list(data['data'].index)))
    
    write_full_res_data(worksheet1, d+st, 3, dtAllRules, data['data']) # Global optimization-Support
    write_full_res_data(worksheet1, d+st, 7, polyAllRules, data['data']) # Poly-Support
    write_full_res_data(worksheet1, d+st, 11, logAllRules, data['data']) # Log-Support
    write_full_res_data(worksheet1, d+st, 15, rmAllRules, data['data']) # RM-Support
    write_full_res_data(worksheet1, d+st, 19, dtAllRules, data['data']) #Decision tree-Support

    cross_val = cross_val_res(data['folds'])
    if d==0:
        dcv=0
    else:
        dcv=dcv+14

    wksht_cross_val.write(dcv+0, 0, 'Dataset')
    wksht_cross_val.write(dcv+0, 1, 'Global accuracy-Length')
    wksht_cross_val.write(dcv+0, 2, 'Poly')
    wksht_cross_val.write(dcv+0, 3, 'Log')
    wksht_cross_val.write(dcv+0, 4, 'RM')
    wksht_cross_val.write(dcv+0, 5, 'Decision tree')
    
    wksht_cross_val.write(dcv+len(cross_val)+2, 0, 'Dataset')
    wksht_cross_val.write(dcv+len(cross_val)+2, 1, 'Global accuracy-Support')
    wksht_cross_val.write(dcv+len(cross_val)+2, 2, 'Poly')
    wksht_cross_val.write(dcv+len(cross_val)+2, 3, 'Log')
    wksht_cross_val.write(dcv+len(cross_val)+2, 4, 'RM')
    wksht_cross_val.write(dcv+len(cross_val)+2, 5, 'Decision tree')
    
    for ci, cv in enumerate(cross_val):
        wksht_cross_val.write(dcv+ci+1, 0, data['name'])
        wksht_cross_val.write(dcv+ci+1, 1, cv['global_Len'])
        wksht_cross_val.write(dcv+ci+1, 2, cv['poly'])
        wksht_cross_val.write(dcv+ci+1, 3, cv['log'])
        wksht_cross_val.write(dcv+ci+1, 4, cv['rm'])
        wksht_cross_val.write(dcv+ci+1, 5, cv['dt'])

        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 0, data['name'])
        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 1, cv['global_Supp'])
        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 2, cv['poly'])
        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 3, cv['log'])
        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 4, cv['rm'])
        wksht_cross_val.write(dcv+len(cross_val)+2+ci+1, 5, cv['dt'])
    
    df_mean_std = pd.DataFrame(cross_val)

    # Calculate the mean and std of each key
    mean_values = df_mean_std.mean().to_dict()
    std_values = df_mean_std.std().to_dict()

    full_results_accur_header(worksheet1, len(datas)+20)
    st = len(datas)+21

    worksheet1.write(d+st, 0, data['name'])
    worksheet1.write(d+st, 1, mean_values['global_Supp'])
    worksheet1.write(d+st, 2, std_values['global_Supp'])
    worksheet1.write(d+st, 3, mean_values['poly'])
    worksheet1.write(d+st, 4, std_values['poly'])
    worksheet1.write(d+st, 5, mean_values['log'])
    worksheet1.write(d+st, 6, std_values['log'])
    worksheet1.write(d+st, 7, mean_values['rm'])
    worksheet1.write(d+st, 8, std_values['rm'])
    worksheet1.write(d+st, 9, mean_values['dt'])
    worksheet1.write(d+st, 10, std_values['dt'])

    full_results_accur_header(worksheet1, len(datas)+35)
    st = len(datas)+36
    worksheet1.write(d+st, 0, data['name'])
    worksheet1.write(d+st, 1, mean_values['global_Len'])
    worksheet1.write(d+st, 2, std_values['global_Len'])
    worksheet1.write(d+st, 3, mean_values['poly'])
    worksheet1.write(d+st, 4, std_values['poly'])
    worksheet1.write(d+st, 5, mean_values['log'])
    worksheet1.write(d+st, 6, std_values['log'])
    worksheet1.write(d+st, 7, mean_values['rm'])
    worksheet1.write(d+st, 8, std_values['rm'])
    worksheet1.write(d+st, 9, mean_values['dt'])
    worksheet1.write(d+st, 10, std_values['dt'])
    print(data['name'])

workbook1.close()

balance-scale
breast-cancer
cars
flags
house-votes
lymphography
mushroom
nursery
tic-tac-toe


In [66]:
# Read the CSV file
df_cv = pd.read_csv('classification_results.csv')

# Initialize a list to hold the data for each dataset
datasets = []

# Extract and organize the data for each dataset
for dataset, group in df_cv.groupby('Dataset'):
    algorithm_A_accuracy = group['Global accuracy-Length'].tolist()
    algorithm_B_accuracy = group['Decision tree'].tolist()
    dataset_dict = {
        'Dataset': dataset,
        'Global Optimization': algorithm_A_accuracy,
        'Decision Tree': algorithm_B_accuracy
    }
    datasets.append(dataset_dict)



In [75]:
# Perform the Wilcoxon signed-rank test for each dataset
alpha = 0.05  # Set the significance level
results = []

for dataset in datasets:
    algorithm_A_accuracy = dataset['Global Optimization']
    algorithm_B_accuracy = dataset['Decision Tree']
    statistic, p_value = wilcoxon(algorithm_A_accuracy, algorithm_B_accuracy)
    
    result_dict = {
        'Dataset': dataset['Dataset'],
        'Wilcoxon Test Statistic': statistic,
        'alpha': alpha,
        'P-value': p_value,
        'Significant Difference': 'Yes' if p_value < alpha else 'No'
    }
    results.append(result_dict)

# Create a DataFrame and display the results in a tabular form
results_df = pd.DataFrame(results)
display(results_df)
results_df.to_csv('results_df.csv')

C:\Users\etetteh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
C:\Users\etetteh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,Dataset,Wilcoxon Test Statistic,alpha,P-value,Significant Difference
0,balance-scale,0.0,0.05,0.062500,No
1,cars,6.0,0.05,0.812500,No
2,flags,4.5,0.05,0.625000,No
3,house-votes,0.0,0.05,0.065600,No
4,lymphography,3.0,0.05,0.465209,No
5,nursery,0.0,0.05,0.062500,No
6,tic-tac-toe,0.0,0.05,0.067889,No
